# Week 3 – Portfolio Optimization in PyTorch

## Problem Setup

We study a **dynamic portfolio optimization** problem. At each rebalancing time \(t\), we observe returns for \(N_t\) assets and choose a portfolio weight vector
\[
w_t \in \mathbb{R}^{N_t}
\]
using only information available up to time \(t\). The portfolio is then held over \([t, t+1]\), and its realized return is
\[
r^{\text{port}}_{t+1} = w_t^\top r_{t+1},
\]
where \(r_{t+1}\) is the vector of realized asset returns.

### Mathematical formulation

Given an estimate of conditional expected returns \(\mu_t \approx \mathbb{E}_t[r_{t+1}]\) and a conditional covariance matrix \(\Sigma_t \approx \mathrm{Var}_t(r_{t+1})\), we solve a **mean–variance** problem with a turnover penalty:
\[
\max_{w_t} \; \mu_t^\top w_t
    - \gamma\, w_t^\top \Sigma_t w_t
    - \kappa \,\lVert w_t - w_{t-1} \rVert_1
\]
subject to
\[
w_{t,i} \ge 0 \quad \forall i, \qquad \sum_{i=1}^{N_t} w_{t,i} = 1.
\]

- \(\gamma > 0\): risk-aversion (penalizes variance).
- \(\kappa \ge 0\): turnover penalty (proxy for transaction costs).
- Constraints: **long-only, fully invested** portfolio.

### Data requirements

- A monthly CRSP-style panel with at least:
  - `date`: month identifier,
  - `permno` or ticker: asset ID,
  - `ret`: monthly excess or total returns.
- In practice, we restrict to a manageable universe (e.g. a fixed subset of large, liquid stocks) so that covariance estimation and PyTorch optimization remain cheap.

### Success metrics

We judge success by **out-of-sample** performance relative to a simple benchmark (e.g. equal-weight portfolio):

- Average monthly return and annualized volatility,
- Annualized Sharpe ratio (using a simple risk-free proxy or \(r_f = 0\)),
- Cumulative return curve,
- Maximum drawdown,
- Turnover and distance to equal-weight (stability of weights).


In [2]:
# Usually all of these are already in Colab, but just in case:
# !pip install pandas numpy torch matplotlib

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

np.random.seed(0)
torch.manual_seed(0)
DEVICE = "cpu"


In [3]:
# Uninstall current pandas installation
!pip uninstall pandas -y

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2


In [4]:
# Reinstall pandas (compatible version)
!pip install pandas==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 48.3 MB/s eta 0:00:00


After running the above two cells, please restart your Colab runtime (Runtime -> Restart runtime...) and then run all cells from the beginning.

In [ ]:
from google.colab import files

uploaded = files.upload()  # choose `sp500_monthly (1).csv` in the dialog

# Get the actual uploaded filename (key of the dict)
filename = list(uploaded.keys())[0]
print("Loaded file:", filename)

df = pd.read_csv(filename)

# Basic checks
print(df.columns)
df.head()


In [ ]:
# Parse dates and sort
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['date', 'permno'])

# Optional: keep only common shares on main exchanges (standard CRSP filters)
if 'shrcd' in df.columns:
    df = df[df['shrcd'].isin([10, 11])]
if 'exchcd' in df.columns:
    df = df[df['exchcd'].isin([1, 2, 3])]

# Drop missing returns
df = df.dropna(subset=['ret'])

print("Unique dates:", df['date'].nunique())
print("Unique permnos:", df['permno'].nunique())

# Pick a stable small universe: permnos with the longest history
counts = df.groupby('permno')['date'].nunique().sort_values(ascending=False)
K_UNIVERSE = 30   # you can tweak this (20–50 is fine for Week 3)
universe_permnos = counts.head(K_UNIVERSE).index.tolist()

df_sub = df[df['permno'].isin(universe_permnos)].copy()

# Date x permno matrix of returns
ret_panel = df_sub.pivot(index='date', columns='permno', values='ret').sort_index()

# Drop any dates with missing returns in this universe (simplest approach)
ret_panel = ret_panel.dropna(how='any')

print("Panel shape (T x N):", ret_panel.shape)
ret_panel.head()


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# For reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Define the device for PyTorch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Hyperparameters
ROLLING_WINDOW = 24   # months to estimate mu, Sigma
GAMMA = 5.0           # risk aversion
KAPPA = 0.1           # turnover penalty strength
LR = 0.1              # learning rate
STEPS = 200           # gradient steps per rebalance

dates = ret_panel.index.to_list()
permnos = ret_panel.columns.to_list()
N = len(permnos)

print(f"#dates in panel: {len(dates)}, #assets in universe: {N}")


def estimate_moments(window_returns: pd.DataFrame):
    """
    window_returns: (T_window x N) DataFrame of past returns
    Returns:
        mu_t : (N,) mean vector
        Sigma_t : (N, N) covariance matrix
    """
    mu_t = window_returns.mean(axis=0).values
    Sigma_t = window_returns.cov().values

    # Ensure symmetry and add small jitter for stability
    Sigma_t = 0.5 * (Sigma_t + Sigma_t.T)
    eps = 1e-6
    Sigma_t = Sigma_t + eps * np.eye(Sigma_t.shape[0])
    return mu_t, Sigma_t


def optimize_weights(mu_t, Sigma_t, w_prev=None):
    """
    Solve:
      max_w mu_t^T w - gamma w^T Sigma w - kappa ||w - w_prev||_1
      s.t. w in simplex
    using logits + softmax in PyTorch.
    """
    mu_t_t = torch.tensor(mu_t, dtype=torch.float32, device=DEVICE)
    Sigma_t_t = torch.tensor(Sigma_t, dtype=torch.float32, device=DEVICE)

    if w_prev is None:
        w_prev_np = np.ones(N) / N
    else:
        w_prev_np = w_prev

    w_prev_t = torch.tensor(w_prev_np, dtype=torch.float32, device=DEVICE)

    # Initialize logits close to previous weights
    v = torch.log(w_prev_t + 1e-8)
    v = v.detach().clone().requires_grad_(True)

    optimizer = torch.optim.SGD([v], lr=LR)

    for _ in range(STEPS):
        optimizer.zero_grad()
        w = torch.softmax(v, dim=0)

        exp_ret = (mu_t_t * w).sum()
        var = w @ Sigma_t_t @ w
        turnover = torch.abs(w - w_prev_t).sum()

        objective = exp_ret - GAMMA * var - KAPPA * turnover
        loss = -objective
        loss.backward()
        optimizer.step()

    w_opt = torch.softmax(v, dim=0).detach().cpu().numpy()
    return w_opt

## Implementation Overview

Implementation is organized as follows:

1. **Data loading & panel construction**
   - Load a CRSP-style monthly stock panel.
   - Pivot into a returns matrix with shape `(T, N)` where `T` is the number of months and `N` the number of assets.
   - Align dates across assets and (optionally) drop months with missing data for simplicity.

2. **Moment estimation**
   - For each rebalance date \(t\), use a rolling window of the last `ROLLING_WINDOW` months to estimate:
     - \(\mu_t\): column-wise mean returns,
     - \(\Sigma_t\): sample covariance matrix (symmetrized, with small diagonal jitter for numerical stability).

3. **Optimization in PyTorch**
   - Parameterize weights using logits \(v_t \in \mathbb{R}^N\) and map to the simplex via
     \[
     w_t = \text{softmax}(v_t),
     \]
     which enforces \(w_t \ge 0\) and \(\sum_i w_{t,i} = 1\).
   - Implement the objective
     \[
     \mu_t^\top w_t - \gamma\, w_t^\top \Sigma_t w_t - \kappa \lVert w_t - w_{t-1} \rVert_1
     \]
     as a differentiable function in PyTorch.
   - Optimize over \(v_t\) using a first-order optimizer (e.g. SGD or Adam) for a fixed number of steps.

4. **Backtest loop**
   - For each \(t\) in the test period:
     - Estimate \((\mu_t, \Sigma_t)\) from past data only.
     - Solve for \(w_t\), record it.
     - Compute realized return \(r^{\text{port}}_{t+1} = w_t^\top r_{t+1}\).
   - In parallel, construct an **equal-weight benchmark** rebalanced monthly.

5. **Evaluation**
   - Compute performance statistics (mean, volatility, Sharpe, drawdown, cumulative return).
   - Plot equity curves and examine turnover / distance to equal-weight.


In [ ]:
portfolio_returns = []
benchmark_returns = []
weight_history = []

w_prev = None

for t_idx in range(ROLLING_WINDOW, len(dates) - 1):
    # estimation window: previous ROLLING_WINDOW months
    window_dates = dates[t_idx - ROLLING_WINDOW : t_idx]
    date_t = dates[t_idx]
    date_next = dates[t_idx + 1]

    window_ret = ret_panel.loc[window_dates]
    mu_t, Sigma_t = estimate_moments(window_ret)

    # Optimize at time t
    w_t = optimize_weights(mu_t, Sigma_t, w_prev=w_prev)
    w_prev = w_t
    weight_history.append((date_t, w_t))

    # Realized return next month
    r_next = ret_panel.loc[date_next].values
    port_ret = np.dot(w_t, r_next)

    # Equal-weight benchmark rebalanced monthly
    ew = np.ones(N) / N
    bench_ret = np.dot(ew, r_next)

    portfolio_returns.append(port_ret)
    benchmark_returns.append(bench_ret)

len(portfolio_returns), len(benchmark_returns)


In [ ]:
def performance_metrics(returns, freq=12):
    returns = np.asarray(returns)
    mean_ret = returns.mean()
    std_ret = returns.std(ddof=1)

    if std_ret > 0:
        sharpe = np.sqrt(freq) * mean_ret / std_ret
    else:
        sharpe = np.nan

    vol_annual = std_ret * np.sqrt(freq)

    equity = (1 + returns).cumprod()
    running_max = np.maximum.accumulate(equity)
    drawdown = equity / running_max - 1.0
    max_dd = drawdown.min()

    total_cum_ret = equity[-1] - 1.0

    return {
        "mean_ret_monthly": mean_ret,
        "vol_annual": vol_annual,
        "sharpe_annual": sharpe,
        "max_drawdown": max_dd,
        "cum_return": total_cum_ret,
        "equity_curve": equity,
    }

port_ret_arr = np.array(portfolio_returns)
bench_ret_arr = np.array(benchmark_returns)

metrics_port = performance_metrics(port_ret_arr, freq=12)
metrics_bench = performance_metrics(bench_ret_arr, freq=12)

print("Optimized portfolio:")
for k, v in metrics_port.items():
    if k != "equity_curve":
        print(f"  {k}: {v:.4f}")

print("\nEqual-weight benchmark:")
for k, v in metrics_bench.items():
    if k != "equity_curve":
        print(f"  {k}: {v:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(metrics_port["equity_curve"], label="Optimized portfolio")
plt.plot(metrics_bench["equity_curve"], label="Equal-weight benchmark", linestyle="--")
plt.title("Cumulative Equity Curve")
plt.xlabel("Backtest step")
plt.ylabel("Growth of $1")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
dates_bt = [d for d, _ in weight_history]
W = np.stack([w for _, w in weight_history])  # (T_backtest x N)

print("Weight matrix shape:", W.shape)

avg_weights = W.mean(axis=0)
weight_summary = pd.DataFrame({
    "permno": permnos,
    "avg_weight": avg_weights
}).sort_values("avg_weight", ascending=False)

weight_summary.head(10)


In [ ]:
# --- Validation 1: synthetic sanity check for the optimizer ---

import numpy as np

# Simple synthetic setup:
# 3 assets:
#  - Asset 0: highest mean, highest variance
#  - Asset 2: lowest mean, lowest variance

mu_synth = np.array([0.02, 0.01, 0.005])  # per-period expected returns
sigma_diag = np.array([0.10, 0.05, 0.02])  # per-period vols
Sigma_synth = np.diag(sigma_diag**2)

print("Synthetic mu:", mu_synth)
print("Synthetic diag(Sigma):", np.diag(Sigma_synth))

w_prev_synth = np.ones(3) / 3  # start from equal weight

# Try a very return-seeking gamma
GAMMA_low = 0.1
w_low = optimize_weights(
    mu_synth, Sigma_synth,
    w_prev=w_prev_synth,
    gamma=GAMMA_low,
    kappa=0.0,
)

# Try a very risk-averse gamma
GAMMA_high = 10.0
w_high = optimize_weights(
    mu_synth, Sigma_synth,
    w_prev=w_prev_synth,
    gamma=GAMMA_high,
    kappa=0.0,
)

print("\nLow gamma (return-seeking):")
print("  weights:", np.round(w_low, 4), " sum =", w_low.sum())

print("\nHigh gamma (risk-averse):")
print("  weights:", np.round(w_high, 4), " sum =", w_high.sum())


In [ ]:
# --- Validation 2: performance measurements & resource summary ---

import time

portfolio_returns = np.asarray(portfolio_returns)
benchmark_returns = np.asarray(benchmark_returns)

def compute_stats(r):
    """Return mean, vol (annualized), Sharpe (annualized), max drawdown, cumulative return."""
    r = np.asarray(r)
    mean_r = r.mean()
    vol = r.std(ddof=1)
    vol_ann = vol * np.sqrt(12)  # monthly → annual
    sharpe = (mean_r * 12) / vol_ann if vol_ann > 0 else np.nan

    cum = np.cumprod(1 + r)
    peak = np.maximum.accumulate(cum)
    dd = (cum - peak) / peak
    max_dd = dd.min()
    cum_ret = cum[-1] - 1

    return dict(
        mean=mean_r,
        vol_ann=vol_ann,
        sharpe=sharpe,
        max_drawdown=max_dd,
        cum_return=cum_ret,
    )

stats_port = compute_stats(portfolio_returns)
stats_bench = compute_stats(benchmark_returns)

print("Portfolio stats:")
for k, v in stats_port.items():
    print(f"  {k:>12}: {v:.4f}")

print("\nBenchmark (equal-weight) stats:")
for k, v in stats_bench.items():
    print(f"  {k:>12}: {v:.4f}")

# Basic resource / configuration summary
T_bt = len(portfolio_returns)
N_assets = ret_panel.shape[1]

print("\nBacktest configuration:")
print(f"  Rolling window (months): {ROLLING_WINDOW}")
print(f"  Number of assets:        {N_assets}")
print(f"  Number of backtest steps:{T_bt}")
print(f"  PyTorch device:          {DEVICE}")

# Rough timing of one optimization call (on last window)
start = time.time()
mu_last, Sigma_last = estimate_moments(
    ret_panel.iloc[-ROLLING_WINDOW:]
)
_ = optimize_weights(
    mu_last, Sigma_last,
    w_prev=np.ones(N_assets)/N_assets,
    gamma=GAMMA,
    kappa=KAPPA,
)
elapsed = time.time() - start
print(f"\nApprox. optimize_weights runtime (single call): {elapsed:.4f} seconds")


In [ ]:
# --- Validation 3: simple edge case handling ---

# Edge case: very small window or nearly constant returns
# Use a tiny 3x3 matrix with almost constant rows

edge_returns = np.array([
    [0.01, 0.01, 0.01],
    [0.0101, 0.0099, 0.0100],
    [0.0099, 0.0102, 0.0098],
])

mu_edge, Sigma_edge = estimate_moments(edge_returns)

print("Edge-case mu:", np.round(mu_edge, 6))
print("Edge-case Sigma diag:", np.round(np.diag(Sigma_edge), 8))

w_prev_edge = np.ones(3) / 3
w_edge = optimize_weights(
    mu_edge, Sigma_edge,
    w_prev=w_prev_edge,
    gamma=GAMMA,
    kappa=KAPPA,
)

print("Edge-case weights:", np.round(w_edge, 4), " sum =", w_edge.sum())


## Documentation: Limitations, Debugging, and Next Steps

### Known limitations

- **Naive moment estimates**: \(\mu_t\) and \(\Sigma_t\) are based on simple rolling sample means and covariances with a fixed window. This is noisy in equity data and may not provide strong cross-sectional signals.
- **Heuristic hyperparameters**: The risk-aversion \(\gamma\) and turnover penalty \(\kappa\) are chosen by hand rather than via systematic tuning or validation.
- **Simplified data handling**: To keep implementation simple, we restrict to a fixed universe and drop months with missing returns, which may introduce survivorship bias and ignore realistic frictions.

### Debug / test strategies

To debug and validate the code, I used:

- **Synthetic tests**: A 3-asset synthetic setup where behavior under low vs high \(\gamma\) is easy to interpret (tilt toward high-mean vs low-variance assets).
- **Shape and constraint checks**: Assertions and prints confirming that
  - weight vectors have the correct shape,
  - all weights are nonnegative,
  - weights sum to 1 up to numerical tolerance.
- **Edge-case tests**: A small example with nearly constant returns to ensure that moment estimation and optimization are numerically stable even when covariance is close to singular.

### Next steps

- Replace the rolling mean \(\mu_t\) with a simple predictive signal (e.g. momentum) and test whether optimized portfolios deviate more meaningfully from equal-weight.
- Add basic covariance shrinkage (e.g. toward the identity or a diagonal matrix) to improve stability in \(\Sigma_t\).
- Systematically vary \(\gamma\) and \(\kappa\) and record how Sharpe ratio, drawdown, and turnover change, using small grid search and visualization.
